In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

In [2]:
employees_df = pd.read_csv("employee_data.csv")


In [3]:
display(employees_df.head())

emp_id    employee_name current_domain  \
0  EMP001     Advait Reddy            R&D   
1  EMP002      Arya Kapoor            R&D   
2  EMP003     Aahana Patel            R&D   
3  EMP004  Niyati Raghavan            R&D   
4  EMP005     Aanya Bansal            R&D   

                            current_job  \
0  Materials Science Research Scientist   
1  Materials Science Research Scientist   
2        Strategic Partnerships Manager   
3          Space Exploration Researcher   
4          Space Exploration Researcher   

                                 current_soft_skills  \
0  Analytical Thinking, Laboratory Skills, Materi...   
1  Analytical Thinking, Laboratory Skills, Materi...   
2  Relationship Building, Negotiation, Business D...   
3       Curiosity, Analytical Thinking, Adaptability   
4       Curiosity, Analytical Thinking, Adaptability   

                                  current_tech_stack  years_of_experience  \
0  Materials Testing, Spectroscopy, R&D in Materi...                  9.0   
1  Materials Testing, Spectroscopy, R&D in Materi...                  9.0   
2      Partnership Management Tools, Market Research                  8.0   
3                    Satellite Systems, Astrophysics                  7.0   
4                    Satellite Systems, Astrophysics                  7.0   

      salary      job_recommendation1                   job_recommendation2  \
0  32,50,000  Chief Marketing Officer          Space Exploration Researcher   
1  32,50,000  Chief Marketing Officer          Space Exploration Researcher   
2  32,50,000  Chief Marketing Officer          Space Exploration Researcher   
3  32,50,000  Chief Marketing Officer  Materials Science Research Scientist   
4  32,50,000  Chief Marketing Officer  Materials Science Research Scientist   

                    job_recommendation3  
0        Strategic Partnerships Manager  
1        Strategic Partnerships Manager  
2  Materials Science Research Scientist  
3        Strategic Partnerships Manager  
4        Strategic Partnerships Manager

In [4]:
y = employees_df[['job_recommendation1', 'job_recommendation2', 'job_recommendation3']]


In [5]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

for column in y.columns:
    y[column] = label_encoder.fit_transform(y[column])


<ipython-input-5-d4e50b069503>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[column] = label_encoder.fit_transform(y[column])
<ipython-input-5-d4e50b069503>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[column] = label_encoder.fit_transform(y[column])
<ipython-input-5-d4e50b069503>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

In [6]:
print(y.dtypes)

job_recommendation1    int64
job_recommendation2    int64
job_recommendation3    int64
dtype: object


In [7]:
employees_df['combined_features'] = employees_df['current_soft_skills'] + ' ' + employees_df['current_tech_stack']

In [8]:
# Assuming employees_df is your DataFrame
employees_df.drop(['current_soft_skills', 'current_tech_stack'], axis=1, inplace=True)


In [9]:
display(employees_df.head())

emp_id    employee_name current_domain  \
0  EMP001     Advait Reddy            R&D   
1  EMP002      Arya Kapoor            R&D   
2  EMP003     Aahana Patel            R&D   
3  EMP004  Niyati Raghavan            R&D   
4  EMP005     Aanya Bansal            R&D   

                            current_job  years_of_experience     salary  \
0  Materials Science Research Scientist                  9.0  32,50,000   
1  Materials Science Research Scientist                  9.0  32,50,000   
2        Strategic Partnerships Manager                  8.0  32,50,000   
3          Space Exploration Researcher                  7.0  32,50,000   
4          Space Exploration Researcher                  7.0  32,50,000   

       job_recommendation1                   job_recommendation2  \
0  Chief Marketing Officer          Space Exploration Researcher   
1  Chief Marketing Officer          Space Exploration Researcher   
2  Chief Marketing Officer          Space Exploration Researcher   
3  Chief Marketing Officer  Materials Science Research Scientist   
4  Chief Marketing Officer  Materials Science Research Scientist   

                    job_recommendation3  \
0        Strategic Partnerships Manager   
1        Strategic Partnerships Manager   
2  Materials Science Research Scientist   
3        Strategic Partnerships Manager   
4        Strategic Partnerships Manager   

                                   combined_features  
0  Analytical Thinking, Laboratory Skills, Materi...  
1  Analytical Thinking, Laboratory Skills, Materi...  
2  Relationship Building, Negotiation, Business D...  
3  Curiosity, Analytical Thinking, Adaptability S...  
4  Curiosity, Analytical Thinking, Adaptability S...

In [10]:
employees_df['combined_features'] = employees_df['combined_features'].fillna("")  # Handle NaN values
employees_df['combined_features'] = employees_df['combined_features'].astype(str)  # Convert to string

vectorizer = CountVectorizer()

# Fit and transform the "combined_features" column
combined_features_matrix = vectorizer.fit_transform(employees_df['combined_features'])

# Convert the resulting matrix to a DataFrame and concatenate it with the original DataFrame
combined_features_df = pd.DataFrame(combined_features_matrix.toarray(), columns=vectorizer.get_feature_names_out())
employees_df = pd.concat([employees_df, combined_features_df], axis=1)


In [11]:
display(employees_df.head(16))

emp_id       employee_name current_domain  \
0   EMP001        Advait Reddy            R&D   
1   EMP002         Arya Kapoor            R&D   
2   EMP003        Aahana Patel            R&D   
3   EMP004     Niyati Raghavan            R&D   
4   EMP005        Aanya Bansal            R&D   
5   EMP006         Arnav Reddy      Marketing   
6   EMP007       Advika Thakur      Marketing   
7   EMP008        Aditya Mehra          Sales   
8   EMP009  Advait Chakraborty          Sales   
9   EMP010       Preeti Sharma        Finance   
10  EMP011       Karan Khurana            R&D   
11  EMP012        Kabir Kapoor            R&D   
12  EMP013         Aditi Mehra            R&D   
13  EMP014      Kiara Malhotra            R&D   
14  EMP015       Vihaan Bansal            R&D   
15  EMP016        Aarav Bansal            R&D   

                             current_job  years_of_experience     salary  \
0   Materials Science Research Scientist                  9.0  32,50,000   
1   Materials Science Research Scientist                  9.0  32,50,000   
2         Strategic Partnerships Manager                  8.0  32,50,000   
3           Space Exploration Researcher                  7.0  32,50,000   
4           Space Exploration Researcher                  7.0  32,50,000   
5                Chief Marketing Officer                 15.0  30,00,000   
6                Chief Marketing Officer                 15.0  30,00,000   
7                          Sales Analyst                  5.0  28,12,500   
8                          Sales Analyst                  5.0  28,12,500   
9               Accounts Payable Manager                  8.0  28,00,000   
10         Quantum Information Scientist                  7.0  27,50,000   
11            Product Innovation Manager                  7.0  27,50,000   
12                 Innovation Strategist                  7.0  27,50,000   
13                Innovation Lab Manager                  7.0  27,50,000   
14         Quantum Information Scientist                  7.0  27,50,000   
15                 Innovation Strategist                  7.0  27,50,000   

                  job_recommendation1                   job_recommendation2  \
0             Chief Marketing Officer          Space Exploration Researcher   
1             Chief Marketing Officer          Space Exploration Researcher   
2             Chief Marketing Officer          Space Exploration Researcher   
3             Chief Marketing Officer  Materials Science Research Scientist   
4             Chief Marketing Officer  Materials Science Research Scientist   
5               Innovation Strategist            Product Innovation Manager   
6               Innovation Strategist            Product Innovation Manager   
7          Product Innovation Manager               Chief Marketing Officer   
8          Product Innovation Manager               Chief Marketing Officer   
9   Nanotechnology Research Scientist               Chief Marketing Officer   
10         Quantum Computing Engineer                         Sales Analyst   
11            Chief People Strategist               Chief Marketing Officer   
12            Chief People Strategist               Chief Marketing Officer   
13                   Chief HR Officer                 IT Compliance Manager   
14         Quantum Computing Engineer                         Sales Analyst   
15            Chief People Strategist               Chief Marketing Officer   

                     job_recommendation3  \
0         Strategic Partnerships Manager   
1         Strategic Partnerships Manager   
2   Materials Science Research Scientist   
3         Strategic Partnerships Manager   
4         Strategic Partnerships Manager   
5           Space Exploration Researcher   
6           Space Exploration Researcher   
7               Accounts Payable Manager   
8               Accounts Payable Manager   
9                  Innovation Strategist   
10              E-commerce Sales Manager   
11   

In [12]:
employees_df.drop(['combined_features'], axis=1, inplace=True)

In [13]:
employees_df.drop(['employee_name'], axis=1, inplace=True)
employees_df.drop(['current_domain'], axis=1, inplace=True)

In [14]:
display(employees_df.head())

emp_id                           current_job  years_of_experience  \
0  EMP001  Materials Science Research Scientist                  9.0   
1  EMP002  Materials Science Research Scientist                  9.0   
2  EMP003        Strategic Partnerships Manager                  8.0   
3  EMP004          Space Exploration Researcher                  7.0   
4  EMP005          Space Exploration Researcher                  7.0   

      salary      job_recommendation1                   job_recommendation2  \
0  32,50,000  Chief Marketing Officer          Space Exploration Researcher   
1  32,50,000  Chief Marketing Officer          Space Exploration Researcher   
2  32,50,000  Chief Marketing Officer          Space Exploration Researcher   
3  32,50,000  Chief Marketing Officer  Materials Science Research Scientist   
4  32,50,000  Chief Marketing Officer  Materials Science Research Scientist   

                    job_recommendation3  3d  3pl  accessibility  ...  wind  \
0        Strategic Partnerships Manager   0    0              0  ...     0   
1        Strategic Partnerships Manager   0    0              0  ...     0   
2  Materials Science Research Scientist   0    0              0  ...     0   
3        Strategic Partnerships Manager   0    0              0  ...     0   
4        Strategic Partnerships Manager   0    0              0  ...     0   

   windows  wireless  wlan  wms  workforce  workplace  workshops  writing  xd  
0        0         0     0    0          0          0          0        0   0  
1        0         0     0    0          0          0          0        0   0  
2        0         0     0    0          0          0          0        0   0  
3        0         0     0    0          0          0          0        0   0  
4        0         0     0    0          0          0          0        0   0  

[5 rows x 692 columns]

In [15]:
from sklearn.preprocessing import LabelEncoder

# Assuming df is your DataFrame and 'emp_id' is the column you want to convert
le = LabelEncoder()
employees_df['emp_id_numeric'] = le.fit_transform(employees_df['emp_id'])

In [16]:
display(employees_df.head())

emp_id                           current_job  years_of_experience  \
0  EMP001  Materials Science Research Scientist                  9.0   
1  EMP002  Materials Science Research Scientist                  9.0   
2  EMP003        Strategic Partnerships Manager                  8.0   
3  EMP004          Space Exploration Researcher                  7.0   
4  EMP005          Space Exploration Researcher                  7.0   

      salary      job_recommendation1                   job_recommendation2  \
0  32,50,000  Chief Marketing Officer          Space Exploration Researcher   
1  32,50,000  Chief Marketing Officer          Space Exploration Researcher   
2  32,50,000  Chief Marketing Officer          Space Exploration Researcher   
3  32,50,000  Chief Marketing Officer  Materials Science Research Scientist   
4  32,50,000  Chief Marketing Officer  Materials Science Research Scientist   

                    job_recommendation3  3d  3pl  accessibility  ...  windows  \
0        Strategic Partnerships Manager   0    0              0  ...        0   
1        Strategic Partnerships Manager   0    0              0  ...        0   
2  Materials Science Research Scientist   0    0              0  ...        0   
3        Strategic Partnerships Manager   0    0              0  ...        0   
4        Strategic Partnerships Manager   0    0              0  ...        0   

   wireless  wlan  wms  workforce  workplace  workshops  writing  xd  \
0         0     0    0          0          0          0        0   0   
1         0     0    0          0          0          0        0   0   
2         0     0    0          0          0          0        0   0   
3         0     0    0          0          0          0        0   0   
4         0     0    0          0          0          0        0   0   

   emp_id_numeric  
0               0  
1               1  
2               2  
3               3  
4               4  

[5 rows x 693 columns]

In [17]:
employees_df.drop(['emp_id'], axis=1, inplace=True)

In [18]:
display(employees_df.head())

current_job  years_of_experience     salary  \
0  Materials Science Research Scientist                  9.0  32,50,000   
1  Materials Science Research Scientist                  9.0  32,50,000   
2        Strategic Partnerships Manager                  8.0  32,50,000   
3          Space Exploration Researcher                  7.0  32,50,000   
4          Space Exploration Researcher                  7.0  32,50,000   

       job_recommendation1                   job_recommendation2  \
0  Chief Marketing Officer          Space Exploration Researcher   
1  Chief Marketing Officer          Space Exploration Researcher   
2  Chief Marketing Officer          Space Exploration Researcher   
3  Chief Marketing Officer  Materials Science Research Scientist   
4  Chief Marketing Officer  Materials Science Research Scientist   

                    job_recommendation3  3d  3pl  accessibility  account  ...  \
0        Strategic Partnerships Manager   0    0              0        0  ...   
1        Strategic Partnerships Manager   0    0              0        0  ...   
2  Materials Science Research Scientist   0    0              0        0  ...   
3        Strategic Partnerships Manager   0    0              0        0  ...   
4        Strategic Partnerships Manager   0    0              0        0  ...   

   windows  wireless  wlan  wms  workforce  workplace  workshops  writing  xd  \
0        0         0     0    0          0          0          0        0   0   
1        0         0     0    0          0          0          0        0   0   
2        0         0     0    0          0          0          0        0   0   
3        0         0     0    0          0          0          0        0   0   
4        0         0     0    0          0          0          0        0   0   

   emp_id_numeric  
0               0  
1               1  
2               2  
3               3  
4               4  

[5 rows x 692 columns]

In [19]:
employees_df = employees_df[['emp_id_numeric'] + [col for col in employees_df.columns if col != 'emp_id_numeric']]

In [20]:
display(employees_df.head())

emp_id_numeric                           current_job  years_of_experience  \
0               0  Materials Science Research Scientist                  9.0   
1               1  Materials Science Research Scientist                  9.0   
2               2        Strategic Partnerships Manager                  8.0   
3               3          Space Exploration Researcher                  7.0   
4               4          Space Exploration Researcher                  7.0   

      salary      job_recommendation1                   job_recommendation2  \
0  32,50,000  Chief Marketing Officer          Space Exploration Researcher   
1  32,50,000  Chief Marketing Officer          Space Exploration Researcher   
2  32,50,000  Chief Marketing Officer          Space Exploration Researcher   
3  32,50,000  Chief Marketing Officer  Materials Science Research Scientist   
4  32,50,000  Chief Marketing Officer  Materials Science Research Scientist   

                    job_recommendation3  3d  3pl  accessibility  ...  wind  \
0        Strategic Partnerships Manager   0    0              0  ...     0   
1        Strategic Partnerships Manager   0    0              0  ...     0   
2  Materials Science Research Scientist   0    0              0  ...     0   
3        Strategic Partnerships Manager   0    0              0  ...     0   
4        Strategic Partnerships Manager   0    0              0  ...     0   

   windows  wireless  wlan  wms  workforce  workplace  workshops  writing  xd  
0        0         0     0    0          0          0          0        0   0  
1        0         0     0    0          0          0          0        0   0  
2        0         0     0    0          0          0          0        0   0  
3        0         0     0    0          0          0          0        0   0  
4        0         0     0    0          0          0          0        0   0  

[5 rows x 692 columns]

In [21]:
from sklearn.preprocessing import LabelEncoder

# Assuming df is your DataFrame and 'current_job' is the column you want to convert
le = LabelEncoder()
employees_df['current_job_numeric'] = le.fit_transform(employees_df['current_job'])

<ipython-input-21-316ef9db9063>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  employees_df['current_job_numeric'] = le.fit_transform(employees_df['current_job'])


In [22]:
employees_df['current_job'].dtypes

dtype('O')

In [23]:
employees_df['current_job'].fillna('', inplace=True)
vectorizer = CountVectorizer()
job_matrix = vectorizer.fit_transform(employees_df['current_job'])
from sklearn.feature_extraction.text import CountVectorizer

# Assuming df is your DataFrame and 'current_job' is the column with job titles
vectorizer = CountVectorizer()
job_matrix = vectorizer.fit_transform(employees_df['current_job'])

<ipython-input-23-0586553aab19>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  employees_df['current_job'].fillna('', inplace=True)


In [24]:
display(employees_df.head())

emp_id_numeric                           current_job  years_of_experience  \
0               0  Materials Science Research Scientist                  9.0   
1               1  Materials Science Research Scientist                  9.0   
2               2        Strategic Partnerships Manager                  8.0   
3               3          Space Exploration Researcher                  7.0   
4               4          Space Exploration Researcher                  7.0   

      salary      job_recommendation1                   job_recommendation2  \
0  32,50,000  Chief Marketing Officer          Space Exploration Researcher   
1  32,50,000  Chief Marketing Officer          Space Exploration Researcher   
2  32,50,000  Chief Marketing Officer          Space Exploration Researcher   
3  32,50,000  Chief Marketing Officer  Materials Science Research Scientist   
4  32,50,000  Chief Marketing Officer  Materials Science Research Scientist   

                    job_recommendation3  3d  3pl  accessibility  ...  windows  \
0        Strategic Partnerships Manager   0    0              0  ...        0   
1        Strategic Partnerships Manager   0    0              0  ...        0   
2  Materials Science Research Scientist   0    0              0  ...        0   
3        Strategic Partnerships Manager   0    0              0  ...        0   
4        Strategic Partnerships Manager   0    0              0  ...        0   

   wireless  wlan  wms  workforce  workplace  workshops  writing  xd  \
0         0     0    0          0          0          0        0   0   
1         0     0    0          0          0          0        0   0   
2         0     0    0          0          0          0        0   0   
3         0     0    0          0          0          0        0   0   
4         0     0    0          0          0          0        0   0   

   current_job_numeric  
0                  289  
1                  289  
2                  421  
3                  419  
4                  419  

[5 rows x 693 columns]

In [25]:
employees_df.drop(['current_job'], axis=1, inplace=True)

In [26]:
employees_df.insert(1, 'current_job_numeric', employees_df.pop('current_job_numeric'))

In [27]:
display(employees_df.head())

emp_id_numeric  current_job_numeric  years_of_experience     salary  \
0               0                  289                  9.0  32,50,000   
1               1                  289                  9.0  32,50,000   
2               2                  421                  8.0  32,50,000   
3               3                  419                  7.0  32,50,000   
4               4                  419                  7.0  32,50,000   

       job_recommendation1                   job_recommendation2  \
0  Chief Marketing Officer          Space Exploration Researcher   
1  Chief Marketing Officer          Space Exploration Researcher   
2  Chief Marketing Officer          Space Exploration Researcher   
3  Chief Marketing Officer  Materials Science Research Scientist   
4  Chief Marketing Officer  Materials Science Research Scientist   

                    job_recommendation3  3d  3pl  accessibility  ...  wind  \
0        Strategic Partnerships Manager   0    0              0  ...     0   
1        Strategic Partnerships Manager   0    0              0  ...     0   
2  Materials Science Research Scientist   0    0              0  ...     0   
3        Strategic Partnerships Manager   0    0              0  ...     0   
4        Strategic Partnerships Manager   0    0              0  ...     0   

   windows  wireless  wlan  wms  workforce  workplace  workshops  writing  xd  
0        0         0     0    0          0          0          0        0   0  
1        0         0     0    0          0          0          0        0   0  
2        0         0     0    0          0          0          0        0   0  
3        0         0     0    0          0          0          0        0   0  
4        0         0     0    0          0          0          0        0   0  

[5 rows x 692 columns]

In [28]:
employees_df['salary'] = pd.to_numeric(employees_df['salary'].str.replace(',', ''), errors='coerce')

In [29]:
employees_df['salary'].dtypes

dtype('float64')

In [30]:
display(employees_df.head())

emp_id_numeric  current_job_numeric  years_of_experience     salary  \
0               0                  289                  9.0  3250000.0   
1               1                  289                  9.0  3250000.0   
2               2                  421                  8.0  3250000.0   
3               3                  419                  7.0  3250000.0   
4               4                  419                  7.0  3250000.0   

       job_recommendation1                   job_recommendation2  \
0  Chief Marketing Officer          Space Exploration Researcher   
1  Chief Marketing Officer          Space Exploration Researcher   
2  Chief Marketing Officer          Space Exploration Researcher   
3  Chief Marketing Officer  Materials Science Research Scientist   
4  Chief Marketing Officer  Materials Science Research Scientist   

                    job_recommendation3  3d  3pl  accessibility  ...  wind  \
0        Strategic Partnerships Manager   0    0              0  ...     0   
1        Strategic Partnerships Manager   0    0              0  ...     0   
2  Materials Science Research Scientist   0    0              0  ...     0   
3        Strategic Partnerships Manager   0    0              0  ...     0   
4        Strategic Partnerships Manager   0    0              0  ...     0   

   windows  wireless  wlan  wms  workforce  workplace  workshops  writing  xd  
0        0         0     0    0          0          0          0        0   0  
1        0         0     0    0          0          0          0        0   0  
2        0         0     0    0          0          0          0        0   0  
3        0         0     0    0          0          0          0        0   0  
4        0         0     0    0          0          0          0        0   0  

[5 rows x 692 columns]

In [31]:
from sklearn.preprocessing import LabelEncoder

# Assuming df is your DataFrame and 'current_job' is the column you want to convert
le = LabelEncoder()
employees_df['job_recommendation1_numeric'] = le.fit_transform(employees_df['job_recommendation1'])
employees_df['job_recommendation1'].fillna('', inplace=True)
vectorizer = CountVectorizer()
job_matrix = vectorizer.fit_transform(employees_df['job_recommendation1'])
from sklearn.feature_extraction.text import CountVectorizer

# Assuming df is your DataFrame and 'current_job' is the column with job titles
vectorizer = CountVectorizer()
job_matrix = vectorizer.fit_transform(employees_df['job_recommendation1'])


In [32]:
display(employees_df.head(10))

emp_id_numeric  current_job_numeric  years_of_experience     salary  \
0               0                  289                  9.0  3250000.0   
1               1                  289                  9.0  3250000.0   
2               2                  421                  8.0  3250000.0   
3               3                  419                  7.0  3250000.0   
4               4                  419                  7.0  3250000.0   
5               5                   45                 15.0  3000000.0   
6               6                   45                 15.0  3000000.0   
7               7                  367                  5.0  2812500.0   
8               8                  367                  5.0  2812500.0   
9               9                   10                  8.0  2800000.0   

                 job_recommendation1                   job_recommendation2  \
0            Chief Marketing Officer          Space Exploration Researcher   
1            Chief Marketing Officer          Space Exploration Researcher   
2            Chief Marketing Officer          Space Exploration Researcher   
3            Chief Marketing Officer  Materials Science Research Scientist   
4            Chief Marketing Officer  Materials Science Research Scientist   
5              Innovation Strategist            Product Innovation Manager   
6              Innovation Strategist            Product Innovation Manager   
7         Product Innovation Manager               Chief Marketing Officer   
8         Product Innovation Manager               Chief Marketing Officer   
9  Nanotechnology Research Scientist               Chief Marketing Officer   

                    job_recommendation3  3d  3pl  accessibility  ...  windows  \
0        Strategic Partnerships Manager   0    0              0  ...        0   
1        Strategic Partnerships Manager   0    0              0  ...        0   
2  Materials Science Research Scientist   0    0              0  ...        0   
3        Strategic Partnerships Manager   0    0              0  ...        0   
4        Strategic Partnerships Manager   0    0              0  ...        0   
5          Space Exploration Researcher   0    0              0  ...        0   
6          Space Exploration Researcher   0    0              0  ...        0   
7              Accounts Payable Manager   0    0              0  ...        0   
8              Accounts Payable Manager   0    0              0  ...        0   
9                 Innovation Strategist   0    0              0  ...        0   

   wireless  wlan  wms  workforce  workplace  workshops  writing  xd  \
0         0     0    0          0          0          0        0   0   
1         0     0    0          0          0          0        0   0   
2         0     0    0          0          0          0        0   0   
3         0     0    0          0          0          0        0   0   
4         0     0    0          0          0          0        0   0   
5         0     0    0          0          0          0        0   0   
6         0     0    0          0          0          0        0   0   
7         0     0    0          0          0          0        0   0   
8         0     0    0          0          0          0        0   0   
9         0     0    0          0          0          0        0   0   

   job_recommendation1_numeric  
0                           22  
1                           22  
2                           22  
3                           22  
4                           22  
5                          111  
6                          111  
7                          147  
8                          147  
9                          139  

[10 rows x 693 columns]

In [33]:
le = LabelEncoder()
employees_df['job_recommendation2_numeric'] = le.fit_transform(employees_df['job_recommendation2'])
employees_df['job_recommendation2'].fillna('', inplace=True)
vectorizer = CountVectorizer()
job_matrix = vectorizer.fit_transform(employees_df['job_recommendation2'])
from sklearn.feature_extraction.text import CountVectorizer

# Assuming df is your DataFrame and 'current_job' is the column with job titles
vectorizer = CountVectorizer()
job_matrix = vectorizer.fit_transform(employees_df['job_recommendation2'])

In [34]:
le = LabelEncoder()
employees_df['job_recommendation3_numeric'] = le.fit_transform(employees_df['job_recommendation3'])
employees_df['job_recommendation3'].fillna('', inplace=True)
vectorizer = CountVectorizer()
job_matrix = vectorizer.fit_transform(employees_df['job_recommendation3'])
from sklearn.feature_extraction.text import CountVectorizer

# Assuming df is your DataFrame and 'current_job' is the column with job titles
vectorizer = CountVectorizer()
job_matrix = vectorizer.fit_transform(employees_df['job_recommendation3'])

In [35]:
display(employees_df.head(10))

emp_id_numeric  current_job_numeric  years_of_experience     salary  \
0               0                  289                  9.0  3250000.0   
1               1                  289                  9.0  3250000.0   
2               2                  421                  8.0  3250000.0   
3               3                  419                  7.0  3250000.0   
4               4                  419                  7.0  3250000.0   
5               5                   45                 15.0  3000000.0   
6               6                   45                 15.0  3000000.0   
7               7                  367                  5.0  2812500.0   
8               8                  367                  5.0  2812500.0   
9               9                   10                  8.0  2800000.0   

                 job_recommendation1                   job_recommendation2  \
0            Chief Marketing Officer          Space Exploration Researcher   
1            Chief Marketing Officer          Space Exploration Researcher   
2            Chief Marketing Officer          Space Exploration Researcher   
3            Chief Marketing Officer  Materials Science Research Scientist   
4            Chief Marketing Officer  Materials Science Research Scientist   
5              Innovation Strategist            Product Innovation Manager   
6              Innovation Strategist            Product Innovation Manager   
7         Product Innovation Manager               Chief Marketing Officer   
8         Product Innovation Manager               Chief Marketing Officer   
9  Nanotechnology Research Scientist               Chief Marketing Officer   

                    job_recommendation3  3d  3pl  accessibility  ...  wlan  \
0        Strategic Partnerships Manager   0    0              0  ...     0   
1        Strategic Partnerships Manager   0    0              0  ...     0   
2  Materials Science Research Scientist   0    0              0  ...     0   
3        Strategic Partnerships Manager   0    0              0  ...     0   
4        Strategic Partnerships Manager   0    0              0  ...     0   
5          Space Exploration Researcher   0    0              0  ...     0   
6          Space Exploration Researcher   0    0              0  ...     0   
7              Accounts Payable Manager   0    0              0  ...     0   
8              Accounts Payable Manager   0    0              0  ...     0   
9                 Innovation Strategist   0    0              0  ...     0   

   wms  workforce  workplace  workshops  writing  xd  \
0    0          0          0          0        0   0   
1    0          0          0          0        0   0   
2    0          0          0          0        0   0   
3    0          0          0          0        0   0   
4    0          0          0          0        0   0   
5    0          0          0          0        0   0   
6    0          0          0          0        0   0   
7    0          0          0          0        0   0   
8    0          0          0          0        0   0   
9    0          0          0          0        0   0   

   job_recommendation1_numeric  job_recommendation2_numeric  \
0                           22                          181   
1                           22                          181   
2                           22                          181   
3                           22                          128   
4                           22                          128   
5                          111                          139   
6                          111                          139   
7                          147                           20   
8                          147                           20   
9                          139                           20   

   job_recommendation3_numeric  
0                          191  
1                          191  
2                          141  
3                          

In [36]:
employees_df.drop(['job_recommendation1'], axis=1, inplace=True)

In [37]:
employees_df.drop(['job_recommendation2'], axis=1, inplace=True)
employees_df.drop(['job_recommendation3'], axis=1, inplace=True)

In [38]:
display(employees_df.head())

emp_id_numeric  current_job_numeric  years_of_experience     salary  3d  \
0               0                  289                  9.0  3250000.0   0   
1               1                  289                  9.0  3250000.0   0   
2               2                  421                  8.0  3250000.0   0   
3               3                  419                  7.0  3250000.0   0   
4               4                  419                  7.0  3250000.0   0   

   3pl  accessibility  account  accounting  active  ...  wlan  wms  workforce  \
0    0              0        0           0       0  ...     0    0          0   
1    0              0        0           0       0  ...     0    0          0   
2    0              0        0           0       0  ...     0    0          0   
3    0              0        0           0       0  ...     0    0          0   
4    0              0        0           0       0  ...     0    0          0   

   workplace  workshops  writing  xd  job_recommendation1_numeric  \
0          0          0        0   0                           22   
1          0          0        0   0                           22   
2          0          0        0   0                           22   
3          0          0        0   0                           22   
4          0          0        0   0                           22   

   job_recommendation2_numeric  job_recommendation3_numeric  
0                          181                          191  
1                          181                          191  
2                          181                          141  
3                          128                          191  
4                          128                          191  

[5 rows x 692 columns]

In [39]:
employees_df.insert(4, 'job_recommendation1_numeric', employees_df.pop('job_recommendation1_numeric'))
employees_df.insert(5, 'job_recommendation2_numeric', employees_df.pop('job_recommendation2_numeric'))
employees_df.insert(6, 'job_recommendation3_numeric', employees_df.pop('job_recommendation3_numeric'))

In [40]:
display(employees_df.head())

emp_id_numeric  current_job_numeric  years_of_experience     salary  \
0               0                  289                  9.0  3250000.0   
1               1                  289                  9.0  3250000.0   
2               2                  421                  8.0  3250000.0   
3               3                  419                  7.0  3250000.0   
4               4                  419                  7.0  3250000.0   

   job_recommendation1_numeric  job_recommendation2_numeric  \
0                           22                          181   
1                           22                          181   
2                           22                          181   
3                           22                          128   
4                           22                          128   

   job_recommendation3_numeric  3d  3pl  accessibility  ...  wind  windows  \
0                          191   0    0              0  ...     0        0   
1                          191   0    0              0  ...     0        0   
2                          141   0    0              0  ...     0        0   
3                          191   0    0              0  ...     0        0   
4                          191   0    0              0  ...     0        0   

   wireless  wlan  wms  workforce  workplace  workshops  writing  xd  
0         0     0    0          0          0          0        0   0  
1         0     0    0          0          0          0        0   0  
2         0     0    0          0          0          0        0   0  
3         0     0    0          0          0          0        0   0  
4         0     0    0          0          0          0        0   0  

[5 rows x 692 columns]

In [41]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Assuming your dataframe is named 'df'
X = employees_df.drop(['job_recommendation1_numeric', 'job_recommendation2_numeric', 'job_recommendation3_numeric'], axis=1)
y = employees_df[['job_recommendation1_numeric', 'job_recommendation2_numeric', 'job_recommendation3_numeric']]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [42]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')

# Define the column transformer for preprocessing
categorical_features = X.columns[X.columns.get_loc('3d'):X.columns.get_loc('xd') + 1]
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('num', imputer, X.select_dtypes(include=['number']).columns)
    ],
    remainder='passthrough'
)

In [43]:
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn.ensemble import HistGradientBoostingRegressor

# Change the regressor to HistGradientBoostingRegressor
regressor = HistGradientBoostingRegressor(random_state=42)
multi_output_regressor = MultiOutputRegressor(regressor)
model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', multi_output_regressor)
])

# Train the model
model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['3d', '3pl', 'accessibility', 'account', 'accounting', 'active',
       'acumen', 'ad', 'adaptability', 'administration',
       ...
       'wind', 'windows', 'wireless', 'wlan', 'wms', 'workforce', 'workplace',
       'workshops', 'writing', 'xd'],
      dtype='o...
                                                  Index(['emp_id_numeric', 'current_job_numeric', 'years_of_experience',
       'salary', '3d', '3pl', 'accessibility', 'account', 'accounting',
       'active',
       ...
       'wind', 'windows', 'wireless', 'wlan', 'wms', 'workforce', 'workplace',
       'workshops', 'writing', 'xd'],
      dtype='object', length=689))])),
                ('regressor',
                 MultiOutputRegressor(estimator=HistGradientBoostingRegressor(random_state=42)))])

In [44]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')
print(f'R-squared: {r2}')

Mean Squared Error: 2444.5772761229587
Mean Absolute Error: 37.52416202462622
R-squared: 0.43141757385895046


In [45]:
# Assuming 'X_test' is your test set
test_predictions = model.predict(X_test)

# Display the predictions for each target variable
for i, target_variable in enumerate(['job_recommendation1_numeric', 'job_recommendation2_numeric', 'job_recommendation3_numeric']):
    print(f'Predictions for {target_variable}: {test_predictions[:, i]}')


Predictions for job_recommendation1_numeric: [ 57.9908109   38.77237925 168.99373772 142.52718851 122.86360036
  85.06238591 166.13323068 124.28892016  45.4152809  162.56637515
  42.3063573   93.54702813  82.8582068  138.52686148 169.9869274
 144.82786297 132.28040008 208.0164219  122.05623328  62.99525263
 211.60361166 168.81311852 135.66010441 105.52060534 134.94205595
  61.50862646  81.39369698 170.36281787  64.23184495  95.2759997
 137.07090357  83.40919111  72.02802917  85.54500678 119.13227059
  76.1984683   67.21191753 113.65546828  39.08507775 151.98794203
 157.91909691 129.94780551  35.45172999  82.54101029  79.26395477
 167.34236782  95.38498996 113.87666426 102.53072566 153.35020414
 135.13660894 159.99108342  82.28040914 195.53203281 164.53936785
  51.09127385 211.60361166  74.03537966 189.5701499  169.9869274
  67.88719531  99.35540415  52.95311301  29.91137507  61.89026934
  86.61169263 100.59609534 127.94698239 146.78076461 114.3389937
  65.51981798 124.54548638  93.9913

In [46]:
for i, target_variable in enumerate(['job_recommendation1_numeric', 'job_recommendation2_numeric', 'job_recommendation3_numeric']):
    print(f'Actual values for {target_variable}: {y_test[target_variable].values}')

Actual values for job_recommendation1_numeric: [ 49   8 206  32 197  37  55 161  16 153   8  49 206  20 100 193  57 191
 150  12 211 206 199 118 159  27 173 158  50 118 173 206  47  57 138  84
  36   7  47 168 181  47   6  71  71 210  35   4  83 124 203 199 121 199
 187 187 211  55 199 100  67  97  44  26  69  21  19 166 197 209  33  28
 210 181  71 162 106  47  66  47 107  20  65  33  50   1 112   2  93 210
   2  28 211 156  43  50 188   1 206  95 121  84  41 106 186  43 149  18
 187  43 156   4  43  43 187   2 206 194   0 206 181 160  47  11 173  18
 199  57 172 176  50 185 206  40  50  57  57 112  37  18  72  94 178  66
 199 168 148  55  57 191 211  78  80 132 132   8 129  28 180 155  95  55
  51 159  89 171  19 198  87 180 111 159 177  25 119 206  95  51 200  43
 181 170  98]
Actual values for job_recommendation2_numeric: [200 182  75  19 196 133  57 115  31 158  73  70  47  29  78  14  88 156
 146 189 207  86 203 151 158 126  17   5  75  56 134  47 193 193 133  18
 185 170 193 110